<div dir="rtl" style="line-height: 1.9">
<h1>عرفان شیرمحمدی</h1>
</div>

In [ ]:
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import IPython
from collections import defaultdict

model = YOLO("yolov8n-seg.pt")  # segmentation model

<div dir="rtl" style="line-height: 1.9">
  <h1>YOLO</h1>
  <p>"you look only once" الگوریتم YOLO تصویر ورودی را به شبکه‌ای از سلول‌ها تقسیم می‌کند و برای هر سلول، احتمال وجود یک شی و مختصات جعبه مرزی شی را  از طریق مدل CNN استخراج میکند و پیش‌بینی می‌کن</p>
  <p><img src="../data/Q.png"></p>
  <p>نقاط قوت : </p>
  <li>سرعت بالا : الگوریتم yolo نسبت به سایر الگوریتم ها تشخیص و ردایابی اشیا سرعت بسیار بیشتری دارد و در برنامه های realtime استفاده میشود مانند ماشین های خودران و یا سیستم های امنیتی
  <li>دقت بالا : این الگوریتم نسبت به سرعت بالای ان دقت خوبی در تشخیص اشیا دارد 
  <li>فریم ورک یکپارجه : این الگوریتم طبقه بندیی و پیش بینی را در یک شبکه ادغام میکند و فرایند اموزش را اسان میکند
  <p>نقاط ضعف : </p>
  <li>دقت : به دلیل سرعت بالای تشخیص دقت مدل کاهش میابد
  <li>این مدل توانایی پایینی در تشخیص اجسام کوچک دارد

</div>

In [1]:
IPython.display.Video(url="../data/car.mp4")

In [ ]:
track_history = defaultdict(lambda: [])
cap = cv2.VideoCapture("../data/car.mp4")

w, h, fps = (
    int(cap.get(x))
    for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS)
)

out = cv2.VideoWriter(
    "../data/car_test_yolov8.avi",
    cv2.VideoWriter_fourcc(*"MJPG"),
    fps,
    (w, h),
)

while True:
    ret, im0 = cap.read()
    if not ret:
        print(
            "Video frame is empty or video processing has been successfully completed."
        )
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)

    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for mask, track_id in zip(masks, track_ids):
            annotator.seg_bbox(
                mask=mask, mask_color=colors(track_id, True), track_label=str(track_id)
            )

    out.write(im0)

    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break

out.release()
cap.release()